In [ ]:
#@title 📦 Download and install rclone binary automatically
import os
import zipfile
import shutil

# Download latest rclone release for Linux amd64
url = "https://downloads.rclone.org/rclone-current-linux-amd64.zip"
zip_path = "/content/rclone.zip"
extract_dir = "/content/rclone_extract"

# Download and unzip
print("⬇️ Downloading rclone...")
!curl -L -o "{zip_path}" "{url}"

print("📦 Extracting...")
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Move the rclone binary to ~/.local/bin
bin_path = os.path.expanduser("~/.local/bin")
os.makedirs(bin_path, exist_ok=True)

# Find the extracted rclone binary path
extracted_dir = next((d for d in os.listdir(extract_dir) if d.startswith("rclone-")), None)
rclone_binary_src = os.path.join(extract_dir, extracted_dir, "rclone")
rclone_binary_dest = os.path.join(bin_path, "rclone")

# Copy and set executable
shutil.copy2(rclone_binary_src, rclone_binary_dest)
os.chmod(rclone_binary_dest, 0o755)

# Update PATH for current session
%env PATH=/root/.local/bin:$PATH

# Confirm installation
print("✅ Rclone installed successfully at ~/.local/bin/rclone\n")
!rclone version


In [2]:
#@title Install rclone (custom binary for filen)
!wget -O rclone https://file-2link-f142f266f968.herokuapp.com/dl/6811caee9e362fa6da3f0437
!chmod +x rclone
!mv rclone /usr/local/bin/rclone
!rclone version


--2025-04-30 11:53:24--  https://file-2link-f142f266f968.herokuapp.com/dl/6811caee9e362fa6da3f0437
Resolving file-2link-f142f266f968.herokuapp.com (file-2link-f142f266f968.herokuapp.com)... 54.78.134.111, 54.228.42.199, 34.241.115.67
Connecting to file-2link-f142f266f968.herokuapp.com (file-2link-f142f266f968.herokuapp.com)|54.78.134.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89373104 (85M) [application/x-executable]
Saving to: ‘rclone’

rclone              100%[===================>]  85.23M  4.14MB/s    in 44s     

2025-04-30 11:54:09 (1.92 MB/s) - ‘rclone’ saved [89373104/89373104]

=== filen-rclone ===
rclone v1.70.0-DEV
- os/version: ubuntu 22.04 (64 bit)
- os/kernel: 6.1.123+ (x86_64)
- os/type: linux
- os/arch: amd64
- go/version: go1.24.2
- go/linking: dynamic
- go/tags: none


In [3]:
#@title Upload your rclone.conf file
from google.colab import files
import os

print("📁 Upload your rclone.conf (found locally at ~/.config/rclone/rclone.conf)")
uploaded = files.upload()

config_dir = "/root/.config/rclone"
os.makedirs(config_dir, exist_ok=True)

for name in uploaded:
    if name.endswith(".conf"):
        os.rename(name, os.path.join(config_dir, "rclone.conf"))
        print(f"✔️ Uploaded and moved {name} to {config_dir}/rclone.conf")


📁 Upload your rclone.conf (found locally at ~/.config/rclone/rclone.conf)


Saving rclone.conf to rclone.conf
✔️ Uploaded and moved rclone.conf to /root/.config/rclone/rclone.conf


In [5]:
!rclone copy photos: filen_35: \
  --ignore-existing \
  --checksum \
  --progress \
  --stats=10s \
  --stats-one-line \
  --log-level=NOTICE \
  --retries=10 \
  --low-level-retries=20 \
  --retries-sleep=30s \
  --cutoff-mode=soft \
  --checkers=4 \
  --transfers=2

=== filen-rclone ===

2025/04/30 11:55:10 ERROR : upload: error reading source directory: directory not found


In [ ]:
#@title Install yt-dlp and required dependencies
!pip install -U yt-dlp
!apt-get install -y ffmpeg


In [ ]:
#@title Prompt to choose rclone remote for upload
import subprocess

# List available rclone remotes
remotes = subprocess.check_output(['rclone', 'listremotes']).decode('utf-8').splitlines()

# Ask user for their choice of remote
remote = input(f"Choose your rclone remote from the following: {', '.join(remotes)}\n")

# Validate the input
if remote not in remotes:
    raise ValueError("Invalid remote selected.")


In [ ]:
#@title Config for videos
import yt_dlp

def download_video(url, remote):
    # yt-dlp options to download the best quality video up to 1080p and audio, merge them
    ydl_opts = {
        'format': 'bestaudio/best[height<=1080]',
        'outtmpl': '/tmp/%(title)s.%(ext)s',  # Temporary download location
        'postprocessors': [{
            'key': 'FFmpegVideoConvertor',
            'preferredformat': 'mp4',  # Convert to mp4 format
        }],
        'merge_output_format': 'mp4',  # Ensure audio and video are merged as mp4
        'noplaylist': True,  # If a playlist is given, treat as individual videos
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        file_path = f"/tmp/{info['title']}.mp4"
        print(f"Downloaded: {file_path}")

        # Upload to rclone remote
        subprocess.run(['rclone', 'copy', file_path, f'{remote}:/path/to/remote/'])
        print(f"Uploaded {file_path} to {remote}")


In [ ]:
#@title Config for playlists
def download_playlist(url, remote):
    # yt-dlp options to download the best quality video up to 1080p and audio, merge them
    ydl_opts = {
        'format': 'bestaudio/best[height<=1080]',
        'outtmpl': '/tmp/%(title)s.%(ext)s',  # Temporary download location
        'postprocessors': [{
            'key': 'FFmpegVideoConvertor',
            'preferredformat': 'mp4',  # Convert to mp4 format
        }],
        'merge_output_format': 'mp4',  # Ensure audio and video are merged as mp4
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        # Iterate through each video in the playlist and upload
        for entry in info['entries']:
            file_path = f"/tmp/{entry['title']}.mp4"
            print(f"Downloaded: {file_path}")

            # Upload to rclone remote
            subprocess.run(['rclone', 'copy', file_path, f'{remote}:/path/to/remote/'])
            print(f"Uploaded {file_path} to {remote}")


In [ ]:
#@title Enter video link and Download
video_url = input("Enter the video URL: ")
download_video(video_url, remote)


In [ ]:
#@title Enter playlist link and download
playlist_url = input("Enter the playlist URL: ")
download_playlist(playlist_url, remote)


In [ ]:
#@title 🔗 Mount an rclone remote
import os

# Make sure rclone binary and rclone.conf are already uploaded and configured
mount_path = "/content/remote"

# Ask user for remote name (as it appears in rclone.conf)
remote = input("Enter the name of your rclone remote (as in rclone.conf): ")

# Create the mount directory
os.makedirs(mount_path, exist_ok=True)

# Mount the remote
# '--daemon' runs rclone in the background, '--allow-other' lets other processes access it
!rclone mount "{remote}": "{mount_path}" --daemon --allow-other --vfs-cache-mode writes

print(f"✅ Remote '{remote}' mounted at {mount_path}")


In [ ]:
#@title Install dependencies
!python -m pip install --upgrade pip setuptools wheel && python -m pip install lbry-libtorrent && apt install python3-libtorrent > /dev/null 2>&1



In [ ]:
#@title Enter the magnet link
import libtorrent as lt
import time
import datetime
import os

def choose_files(files):
    print("\n🔍 Torrent contains the following files:\n")
    for i, f in enumerate(files):
        print(f"[{i}] {f.path} ({f.size // (1024 * 1024)} MB)")

    selected = input("\nEnter indices of files to download (comma-separated), or 'all' to select all: ")

    if selected.strip().lower() == 'all':
        return list(range(len(files)))
    else:
        try:
            indices = [int(i.strip()) for i in selected.split(',')]
            return indices
        except:
            print("❌ Invalid input. Downloading all files.")
            return list(range(len(files)))

def download_and_upload(remote_name, remote_path):
    ses = lt.session()
    ses.listen_on(6881, 6891)

    # Set no download/upload rate limits for unrestricted speeds
    ses.set_download_rate_limit(0)  # Unlimited download rate
    ses.set_upload_rate_limit(0)  # Unlimited upload rate (this can be kept or disabled)

    # Optional: Limit the number of connections and uploads to optimize performance
    ses.set_max_connections(200)  # Increase max connections
    ses.set_max_uploads(10)  # Limit the number of upload connections (optional)

    params = {
        'save_path': '/content/Downloads/',
        'storage_mode': lt.storage_mode_t(2),
    }

    magnet = input("🔗 Enter magnet link: ").strip()
    handle = lt.add_magnet_uri(ses, magnet, params)
    handle.set_sequential_download(False)
    ses.start_dht()

    print("\n⏳ Fetching metadata...")
    while not handle.has_metadata():
        time.sleep(1)

    torrent_info = handle.get_torrent_info()
    files = torrent_info.files()

    selected_indices = choose_files(files)

    for idx in range(len(files)):
        if idx not in selected_indices:
            handle.file_priority(idx, 0)

    print(f"\n✅ Starting download: {torrent_info.name()}")
    print(datetime.datetime.now())

    while handle.status().state != lt.torrent_status.seeding:
        s = handle.status()
        state_str = ['queued', 'checking', 'downloading metadata',
                     'downloading', 'finished', 'seeding', 'allocating']
        print(f"{s.progress * 100:.2f}% complete (↓ {s.download_rate/1000:.1f} kB/s | ↑ {s.upload_rate/1000:.1f} kB/s | peers: {s.num_peers}) [{state_str[s.state]}]")
        time.sleep(5)

    print(f"\n🎉 Download complete: {handle.name()}")
    print("📤 Uploading to rclone remote...")

    # Remote path like: myremote:path/in/remote
    full_remote_path = f"{remote_name}:{remote_path.rstrip('/')}/{handle.name()}"

    !rclone copy "/content/Downloads/{handle.name()}" "{full_remote_path}" --quiet

    print("✅ Upload finished!")

# Usage
remote = input("🌐 Enter your rclone remote name (e.g., gdrive): ").strip()
rpath = input("📁 Enter path in remote (e.g., Torrents/Anime): ").strip()
download_and_upload(remote, rpath)
